<div style="margin: 0px; padding: 10px; background-color: #43a047;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white; text-align:center"> Time Series Forecasting of DOGECOIN using GREYKITE </h1>
      
<!--       <div style="color:white">I am a square root.</div> -->
<!--       <h1>$\sqrt{4}$</h1> -->
  </div>
</div>

<center><img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/a500025c0abbda3915b6ec7c0723ce36f189eed8/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6c696e6b6564696e2f677265796b6974652f6d61737465722f4c4f474f2d43382e706e67" alt="Greykite" class="center" style = "
  width: 60%;
  height: 60%;
  background-image: url('img_flowers.jpg');
  align: center;
  background-size: 100% 100%;"></center>


<div style="margin: 0px; padding: 10px; background-color: #00897b;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white; text-align:center">Table of Contents</h1>
  </div>
  
  <div style="margin: 0px; padding: 10px; background-color: #ffffff;
        box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
        border-radius:2px; text-align:left">
    <h5><a href="#0">0. References</a></h5>
    <h5><a href="#1">1. Installing the Greykite package</a></h5>
    <h5><a href="#2">2. Re-installing the pandas</a></h5>
    <h5><a href="#3">3. Importing all the required libraries.</a></h5>
    <h5><a href="#4">4. Forecaster</a></h5>
    <h5><a href="#5">5. Plotting the results using the Plotly module</a></h5>
    <h5><a href="#6">6. Performing Grid Search for hyper-parameter optimization</a></h5>
    <h5><a href="#7">7. Plotting the Backtest</a></h5>
    <h5><a href="#8">8. Listing the different evaluation metrics</a></h5>
    <h5><a href="#9">9. Analyse the forecast Dataframe</a></h5>
    <h5><a href="#10">10. Plotting the different componenets</a></h5>
    <h5><a href="#11">11. Printing the summary</a></h5>
    <h5><a href="#12">12. Forecasting for the values for the future time periods</a></h5>
    
  </div>
</div>

<a id='0'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white; text-align:center">0. References 📓</h1>
  </div>
  
  <div style="margin: 0px; padding: 10px; background-color: #ffffff;
        box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
        border-radius:2px; text-align:left">
    <h5><a href=https://linkedin.github.io/greykite/docs/0.1.0/html/index.html>1. Greykite Documentation </a></h5>
  </div>
</div>

<a id='1'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px;
            text-align:center">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white">1. Installing the Greykite package 📗</h1>
  </div>
</div>

In [ ]:
!pip install greykite

<a id='2'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">2. <b>Re-installing the pandas to avoid some runtime error...</b></h1>
  </div>
</div>

In [ ]:
!pip install --force-reinstall pandas==1.2.4

<a id='3'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">3. <b>Importing all the required libraries.</b> </h1>
  </div>
</div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/cryptocurrencypricehistory'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import plotly
import plotly.offline as pyo
import plotly.graph_objs as go

pyo.init_notebook_mode()

from greykite.common.data_loader import DataLoader
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

# Loads dataset into pandas DataFrame

df = pd.read_csv('/kaggle/input/cryptocurrencypricehistory/coin_Dogecoin.csv')


metadata = MetadataParam(
 time_col="Date",   # ----> name of the time column ("date" in example above)
 value_col="High",  # ----> name of the value column ("sessions" in example above)
 freq="D"           # ----> H" for hourly, "D" for daily, "W" for weekly, etc.
                    # ----> Any format accepted by `pandas.date_range`
)

<a id='4'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">4. <b>Forecaster</b></h1>
  </div>
  <div style="color:white;text-align:center">The core module which performs the forecasting of the values for the given time series</div>
</div>

In [ ]:
forecaster = Forecaster()  # Creates forecasts and stores the result
result = forecaster.run_forecast_config(  # result is also stored as `forecaster.forecast_result`.
 df=df,
 config=ForecastConfig(
     model_template=ModelTemplateEnum.SILVERKITE.name,
     forecast_horizon=365,  # forecasts 365 steps ahead
     coverage=0.95,         # 95% prediction intervals
     metadata_param=metadata
 )
)

<a id='5'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">5. <b>Plotting the results using the Plotly module  📈</b></h1>
  </div>
</div>

In [ ]:
ts = result.timeseries
fig = ts.plot()
plotly.io.show(fig)

<a id='6'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">6. <b>Performing Grid Search for hyper-parameter optimization</b></h1>
  </div>
</div>

In [ ]:
grid_search = result.grid_search
cv_results = summarize_grid_search_results(
    grid_search=grid_search,
    decimals=2,
    # The below saves space in the printed output. Remove to show all available metrics and columns.
    cv_report_metrics=None,
    column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"])
# Transposes to save space in the printed output
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results.transpose()

<div style="margin: 0px; padding: 10px; background-color: #ef9a9a ;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h2 style="color:white;text-align:center">All the performance metrics such as Mean Absolute Percentage Error(MAPE), Mean Fit Time, Mean Score Time can be listed.</h2>
  </div>
</div>

<a id='7'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">7. <b>Plotting the Backtest</b></h1>
  </div>
</div>

In [ ]:
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

<div style="margin: 0px; padding: 10px; background-color: #ef9a9a ;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h2 style="color:white;text-align:center">Remarks: The model fails to recognise the sudden changes in the trend.</h2>
  </div>
</div>

<a id='8'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">8. <b>Listing the different evaluation metrics</b></h1>
  </div>

</div>

In [ ]:
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics

<div style="margin: 0px; padding: 10px; background-color: #ef9a9a ;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h2 style="color:white;text-align:center">In the above cell, the metrics for both train and test data are listed.</h2>
  </div>
</div>

In [ ]:
forecast = result.forecast
fig = forecast.plot()
plotly.io.show(fig)

<a id='9'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center">9. <b>Analyse the forecast Dataframe</b></h1>
  </div>
</div>

In [ ]:
forecast.df.head().round(2)

<a id='10'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
    
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center"><b>10. Plotting the different components 📊</b></h1>
  </div>
</div>

In [ ]:
fig = forecast.plot_components()
plotly.io.show(fig)

<a id='11'></a>
<div style="margin: 0px; padding: 10px; background-color: #1e88e5;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center"><b>11. Printing the summary</b></h1>
  </div>
</div>

In [ ]:
summary = result.model[-1].summary()  # -1 retrieves the estimator from the pipeline
print(summary)

In [ ]:
model = result.model
model

<a id='12'></a>
<div style="margin: 0px; padding: 10px; background-color: #1de9b6  ;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center"><b>12. Forecasting for the values for the future time periods 📈</b></h1>
  </div>
</div>

In [ ]:
future_df = result.timeseries.make_future_dataframe(
    periods=4,
    include_history=False)
future_df

In [ ]:
model.predict(future_df)

<a id='2'></a>
<div style="margin: 0px; padding: 10px; background-color: #1de9b6 ;
            box-shadow: 0 2px 4px 0 rgba(0,0,0,0.2);
            border-radius:2px">
  <div style="margin: 0; padding: 0; width: 100%">
      <h1 style="color:white;text-align:center"><b>Hooray!!! That's it... 
          We learnt how to use Greykite library for time series analysis and forecasting. This is my first ever notebook. Kindly upvote 👍 the notebook if you liked the content. Thank you. 🙏</b></h1>
  </div>
</div>